<a href="https://colab.research.google.com/github/xujiaxinmeta/Leah_Xu/blob/main/MinHahDocuments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similar document searching via MinHash and Locality Sensitive Hashing
本练习的目标是了解 minhash 和 LSH 系统如何有效和高效地识别这些实例。

# ***Part I: Preliminaries***
**Part IA: Dataset parsing**

这是生成的 MinHash 签名中的组件数。

对应的也是随机哈希函数的个数

我们需要计算 MinHash。

In [89]:
from google.colab import drive
drive.mount('/content/drive')
import time
import binascii
import random
from bisect import bisect_right
from heapq import heappop, heappush
from __future__ import division
import os
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
numHashes = 10;

In [ ]:
numDocs = 1000

In [46]:

dataFile = "/content/drive/MyDrive/Colab Notebooks/DataMining/data/articles_" + str(numDocs) + ".train"
truthFile = "/content/drive/MyDrive/Colab Notebooks/DataMining/data/articles_" + str(numDocs) + ".truth"

In [39]:
plagiaries = {}
f = open(truthFile, "r")


In [40]:
# For each line of the files...
for line in f:
  
  # Strip the newline character, if present.
  if line[-1] == '\n':
      line = line[0:-1]
      
  docs = line.split(" ")

In [98]:
# Map the two documents to each other.
plagiaries[docs[0]] = docs[1]
plagiaries[docs[1]] = docs[0]

## Convert Documents To Sets of Shingles
The current shingle ID value to assign to the next new shingle we ,encounter. When a shingle gets added to the dictionary, we'll increment this value.

In [42]:
curShingleID = 0

In [76]:
# Create a dictionary of the articles, mapping the article identifier (e.g., 
# "t8470") to the list of shingle IDs that appear in the document.
docsAsShingleSets = {}
# Open the data file.
f = open(dataFile, "r")
docNames = []

t0 = time.time()

totalShingles = 0
for i in range(0, numDocs):
  
  # Read all of the words (they are all on one line) and split them by white
  # space.
  words = f.readline().split(" ")
   # Retrieve the article ID, which is the first word on the line.  
  docID = words[0]
  # Maintain a list of all document IDs.  
  docNames.append(docID)
  del words[0] 
# 'shinglesInDoc' will hold all of the unique shingle IDs present in the 
  # current document. If a shingle ID occurs multiple times in the document,
  # it will only appear once in the set (this is a property of Python sets).
  #重复的单词在集合中只会出现一次
  shinglesInDoc = set()
  # For each word in the document...
  for index in range(0, len(words) - 2):

    # Construct the shingle text by combining three words together.
    shingle = (words[index] + " " + words[index + 1] + " " + words[index + 2]).encode()
     # Hash the shingle to a 32-bit integer.
    
    crc = binascii.crc32(shingle) & 0xffffffff
    
     # Add the hash value to the list of shingles for the current document. 
    # Note that set objects will only add the value to the set if the set 
    # doesn't already contain it. 
    shinglesInDoc.add(crc)
    # Store the completed list of shingles for this document in the dictionary.
  docsAsShingleSets[docID] = shinglesInDoc
  # Count the number of shingles across all documents.
  totalShingles = totalShingles + (len(words) - 2)
  # Close the data file.  
f.close()  
print ('\nShingling ' + str(numDocs) + ' docs took %.2f sec.' % (time.time() - t0))
 
print ('\nAverage shingles per doc: %.2f' % (totalShingles / numDocs))


Shingling 1000 docs took 0.30 sec.

Average shingles per doc: 251.24


## Define Triangle Matrices

In [77]:
# Define virtual Triangle matrices to hold the similarity values. For storing
# similarities between pairs, we only need roughly half the elements of a full
# matrix. Using a triangle matrix requires less than half the memory of a full
# matrix, and can protect the programmer from inadvertently accessing one of
# the empty/invalid cells of a full matrix.
# Calculate the number of elements needed in our triangle matrix
numElems = int(numDocs * (numDocs - 1) / 2)
# Initialize two empty lists to store the similarity values. 
# 'JSim' will be for the actual Jaccard Similarity values. 
# 'estJSim' will be for the estimated Jaccard Similarities found by comparing
# the MinHash signatures.
JSim = [0 for x in range(numElems)]
estJSim = [0 for x in range(numElems)]
# Define a function to map a 2D matrix coordinate into a 1D index.
def getTriangleIndex(i, j):
  # If i == j that's an error.
  if i == j:
    sys.stderr.write("Can't access triangle matrix with i == j")
    sys.exit(1)
  # If j < i just swap the values.
  if j < i:
    temp = i
    i = j
    j = temp
    # Calculate the index within the triangular array.
  # This fancy indexing scheme is taken from pg. 211 of:
  # http://infolab.stanford.edu/~ullman/mmds/ch6.pdf
  # But I adapted it for a 0-based index.
  # Note: The division by two should not truncate, it
  #       needs to be a float. 
  k = int(i * (numDocs - (i + 1) / 2.0) + j - i) - 1
  
  return k

In [80]:
# Calculating the Jaccard similarities gets really slow for large numbers of documents.

if numDocs <= 2500:
#if True:
    print ("\nCalculating Jaccard Similarities...")

    # Time the calculation.
    t0 = time.time()

    # For every document pair...
    for i in range(0, numDocs):
      
      # Print progress every 100 documents.
      if (i % 100) == 0:
        print ("  (" + str(i) + " / " + str(numDocs) + ")")

      # Retrieve the set of shingles for document i.
      s1 = docsAsShingleSets[docNames[i]]
      
      for j in range(i + 1, numDocs):
        # Retrieve the set of shingles for document j.
        s2 = docsAsShingleSets[docNames[j]]
        
        # Calculate and store the actual Jaccard similarity.
        JSim[getTriangleIndex(i, j)] = (len(s1.intersection(s2)) / len(s1.union(s2)))    

    # Calculate the elapsed time (in seconds)
    elapsed = (time.time() - t0)
        
    print ("\nCalculating all Jaccard Similarities took %.2fsec" % elapsed)
# Delete the Jaccard Similarities, since it's a pretty big matrix.    
del JSim


Calculating Jaccard Similarities...
  (0 / 1000)
  (100 / 1000)
  (200 / 1000)
  (300 / 1000)
  (400 / 1000)
  (500 / 1000)
  (600 / 1000)
  (700 / 1000)
  (800 / 1000)
  (900 / 1000)

Calculating all Jaccard Similarities took 12.96sec


## Generate MinHash Signatures

In [82]:
# Time this step.
t0 = time.time()

print ('\nGenerating random hash functions...')

# Record the maximum shingle ID that we assigned.
maxShingleID = 2**32-1

# We need the next largest prime number above 'maxShingleID'.
# I looked this value up here: 
# http://compoasso.free.fr/primelistweb/page/prime/liste_online_en.php
nextPrime = 4294967311


# Our random hash function will take the form of:
#   h(x) = (a*x + b) % c
# Where 'x' is the input value, 'a' and 'b' are random coefficients, and 'c' is
# a prime number just greater than maxShingleID.

# Generate a list of 'k' random coefficients for the random hash functions,
# while ensuring that the same value does not appear multiple times in the 
# list.
def pickRandomCoeffs(k):
  # Create a list of 'k' random values.
  randList = []
  
  while k > 0:
    # Get a random shingle ID.
    randIndex = random.randint(0, maxShingleID) 
  
    # Ensure that each random number is unique.
    while randIndex in randList:
      randIndex = random.randint(0, maxShingleID) 
    
    # Add the random number to the list.
    randList.append(randIndex)
    k = k - 1
    
  return randList


Generating random hash functions...


In [83]:
# For each of the 'numHashes' hash functions, generate a different coefficient 'a' and 'b'.   
coeffA = pickRandomCoeffs(numHashes)
coeffB = pickRandomCoeffs(numHashes)

In [84]:
print ('\nGenerating MinHash signatures for all documents...')


Generating MinHash signatures for all documents...


In [86]:
signatures = []
# Rather than generating a random permutation of all possible shingles, 
# we'll just hash the IDs of the shingles that are *actually in the document*,
# then take the lowest resulting hash code value. This corresponds to the index 
# of the first shingle that you would have encountered in the random order.
# For each document...
for docID in docNames:
  
  # Get the shingle set for this document.
  shingleIDSet = docsAsShingleSets[docID]
  
  # The resulting minhash signature for this document. 
  signature = []
  
  # For each of the random hash functions...
  for i in range(0, numHashes):
    
    # For each of the shingles actually in the document, calculate its hash code
    # using hash function 'i'. 
    
    # Track the lowest hash ID seen. Initialize 'minHashCode' to be greater than
    # the maximum possible value output by the hash.
    minHashCode = nextPrime + 1
    
    # For each shingle in the document...
    for shingleID in shingleIDSet:
      # Evaluate the hash function.
      hashCode = (coeffA[i] * shingleID + coeffB[i]) % nextPrime 
      
      # Track the lowest hash code seen.
      if hashCode < minHashCode:
        minHashCode = hashCode

    # Add the smallest hash code value as component number 'i' of the signature.
    signature.append(minHashCode)
  
  # Store the MinHash signature for this document.
  signatures.append(signature)

# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)
        
print ("\nGenerating MinHash signatures took %.2fsec" % elapsed ) 



Generating MinHash signatures took 117.41sec


## Compare All Signatures

In [87]:
print ('\nComparing all signatures...') 
  
# Creates a N x N matrix initialized to 0.

# Time this step.
t0 = time.time()

# For each of the test documents...
for i in range(0, numDocs):
  # Get the MinHash signature for document i.
  signature1 = signatures[i]
    
  # For each of the other test documents...
  for j in range(i + 1, numDocs):
    
    # Get the MinHash signature for document j.
    signature2 = signatures[j]
    
    count = 0
    # Count the number of positions in the minhash signature which are equal.
    for k in range(0, numHashes):
      count = count + (signature1[k] == signature2[k])
    
    # Record the percentage of positions which matched.    
    estJSim[getTriangleIndex(i, j)] = (count / numHashes)

# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)
        
print ("\nComparing MinHash signatures took %.2fsec" % elapsed)  


Comparing all signatures...

Comparing MinHash signatures took 1.94sec


## Display Similar Document Pairs

In [103]:
# Count the true positives and false positives.
tp = 0
fp = 0
  
threshold = 0.5  
print ("\nList of Document Pairs with J(d1,d2) more than", threshold)
print ("Values shown are the estimated Jaccard similarity and the actual")
print ("Jaccard similarity.\n")
print ("                   Est. J   Act. J")
plagiaries = {}
# For each of the document pairs...
for i in range(0, numDocs):  
  for j in range(i + 1, numDocs):
    # Retrieve the estimated similarity value for this pair.
    estJ = estJSim[getTriangleIndex(i, j)]
    
    # If the similarity is above the threshold...
    if estJ > threshold:
    
      # Calculate the actual Jaccard similarity for validation.
      s1 = docsAsShingleSets[docNames[i]]
      s2 = docsAsShingleSets[docNames[j]]
      J = (len(s1.intersection(s2)) / len(s1.union(s2)))
      
      # Print out the match and similarity values with pretty spacing.
      print ("  %5s --> %5s   %.2f     %.2f" % (docNames[i], docNames[j], estJ, J))
      
      if docsAsShingleSets[docNames[i]] == docNames[j]:   
        tp = tp + 1
      else:
        fp = fp + 1

# Display true positive and false positive counts.
print ("True positives:  " + str(tp) + " / " + str(int(len(plagiaries.keys()) / 2)))
print ("False positives: " + str(fp))


List of Document Pairs with J(d1,d2) more than 0.5
Values shown are the estimated Jaccard similarity and the actual
Jaccard similarity.

                   Est. J   Act. J
   t980 --> t2023   1.00     0.98
  t1088 --> t5015   1.00     0.98
  t1297 --> t4638   1.00     0.98
  t1768 --> t5248   1.00     0.98
  t1952 --> t3495   1.00     0.98
  t2535 --> t8642   1.00     0.98
  t2839 --> t9303   1.00     0.98
  t2957 --> t7111   1.00     0.98
  t3268 --> t7998   1.00     0.98
  t3466 --> t7563   1.00     0.98
True positives:  0 / 0
False positives: 10


In [75]:
f = open(dataFile, "r")
words = f.readline().split(" ")
docID = words[0]
shingle = (words[0] + " " + words[0 + 1] + " " + words[0 + 2]).encode()
crc = binascii.crc32(shingle) & 0xffffffff
#type(shingle)